In [4]:
import re
import math
import numpy as np
import statistics

In [5]:
#Do not run this block,this is just for splitting data into 5 sets for 5fold evaluation
##################################
##################################
##################################
# file_split =0
# file_orig = open('a1_d3.txt', 'r')
# index = 1
# while True:
#     line = file_orig.readline()
#     if not line: 
#         break
#     file_write = open('data'+str(index)+'.txt','a')
#     file_write.write(line)
#     file_write.close()
#     file_split +=1
#     if file_split % 200 ==0:
#         index +=1
# file_orig.close()

In [6]:
stop = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
stop_dict ={}
for i in stop:
    stop_dict[i] = 1

In [7]:
def gen_model2(train):
    words = {}
    y_one_present={}
    y_zero_present={}
    word_count=0
    zero_count=0
    one_count =0 
    train_points=0
    for i in train:
        file = open(i, 'r')
        #file = open('temp.txt', 'r') 
        count = 0
#         word_count=0
#         zero_count=0
#         one_count =0 
#         line_count=0
        while True: 
            # Get next line from file 
            inputlineOri = file.readline()
            inputline = inputlineOri.lower()
            train_points +=1
            # if line is empty 
            # end of file is reached 
            if not inputline: 
                break
            flag =0
            line = re.findall(r'\w+', inputline) 
            #print(line)
            flag = line[len(line)-1]
            temp_dict = {}
            #print(type(flag))
            if int(flag)==1: 
                one_count +=1
                for i in range(0,len(line)-1):
                    if line[i] not in stop_dict.keys():
                        words[line[i]]=1
                        #one_count +=1
                        if line[i] not in temp_dict.keys():
                            if line[i] in y_one_present.keys():
                                y_one_present[line[i]]= y_one_present[line[i]]+1
                            else:
                                y_one_present[line[i]]=1
                            temp_dict[line[i]]=1
                #                 words[line[i]]=1
            else:
                zero_count +=1
                for i in range(0,len(line)-1):
                    
                    if line[i] not in stop_dict.keys(): 
                        words[line[i]]=1
                        #zero_count +=1
                        if line[i] not in temp_dict.keys():
                            if line[i] in y_zero_present.keys():
                                y_zero_present[line[i]]=y_zero_present[line[i]]+1
                            else:
                                y_zero_present[line[i]]=1
                            temp_dict[line[i]]=1
            #                 words[line[i]]=1
    file.close() 
#     print("POS")
#     print(pos)
#     print("NEG")
#     print(neg)
#     print(one_count)
#     print(zero_count)
#     print(line_count)
    
    
    for i in words:
        if i not in y_one_present.keys():
            y_one_present[i]=0
        if i not in y_zero_present.keys():
            y_zero_present[i] = 0
    
    return y_one_present,y_zero_present,one_count,zero_count,words

In [8]:
def calculate_yOne(word_count,pos,neg,one_count,zero_count,string):
    value = math.log(one_count / (one_count+zero_count))
    di = {}
    for i in string:
        di[i]=1

    for i in string:
        if i in word_count:
            value +=math.log((pos[i]+1)/(one_count+1))
    
    
    return value

In [9]:
def calculate_yZero(word_count,pos,neg,one_count,zero_count,string):
    value = math.log(zero_count / (one_count+zero_count))
    di = {}
    for i in string:
        di[i]=1

    for i in string:
        if i in word_count:
            value += math.log((neg[i]+1)/(zero_count+1))
    return value

In [10]:
def evaluate(word_count,test,pos,neg,zero_count,one_count): 
    file = open(test, 'r')
    #file = open('temp.txt', 'r') 
    test_line=0
    precision=0
    recall=0
    correct_pred =0
    true_pos=0
    act_pos =0
    act_neg=0
    while True: 
        # Get next line from file 
        inputlineOri = file.readline()
        inputline=inputlineOri.lower()
        test_line +=1
        # if line is empty 
        # end of file is reached 
        if not inputline: 
            break
        flag =0
        line = re.findall(r'\w+', inputline) 
        #print(line)
        val = line[len(line)-1]
        flag = val
        #print(type(flag))
        vpos = calculate_yOne(word_count,pos,neg,one_count,zero_count,line)
        vneg = calculate_yZero(word_count,pos,neg,one_count,zero_count,line)
        
#         print(test_line)
#         print(vpos)
#         print(vneg)
#         print(flag)
        if(vpos > vneg):
            #print("positive")
            if int(flag) == 1:
                #print("correct")
                correct_pred +=1
                true_pos +=1
            else:
                act_neg+=1
        else:
            #print("negative")
            if int(flag) == 0:
                #print("correct")
                correct_pred +=1
            else:
                act_pos +=1
    file.close() 
    print(correct_pred)
    accuracy = float(correct_pred)/test_line
    print("accuracy"+str(float(correct_pred)/test_line))
    precision = float(true_pos)/(act_neg+true_pos)
    recall = float(true_pos)/(true_pos+act_pos)
    #recall = float(correct_pred)/(test_line)
    f_score = (2*precision*recall)/(precision+recall)
    print("precision"+str(precision))
    print("recall"+str(recall))
    print("f_Score"+str(f_score))
    #return (float(correct_pred)/test_line)
    return accuracy,f_score

In [11]:
train=[]
test=[]
train1 = ['data1.txt','data2.txt','data3.txt','data4.txt']
test1 = 'data5.txt'
train.append(train1)
test.append(test1)
train2 = ['data1.txt','data2.txt','data3.txt','data5.txt']
test2 = 'data4.txt'
train.append(train2)
test.append(test2)
train3 = ['data1.txt','data2.txt','data5.txt','data4.txt']
test3 = 'data3.txt'
train.append(train3)
test.append(test3)
train4 = ['data1.txt','data5.txt','data3.txt','data4.txt']
test4 = 'data2.txt'
train.append(train4)
test.append(test4)
train5 = ['data5.txt','data2.txt','data3.txt','data4.txt']
test5 = 'data1.txt'
train.append(train5)
test.append(test5)

In [12]:
val=0
acc=0
val_list =[]
acc_list=[]
for i in range(0,5):
    pos,neg,one_count,zero_count,words = gen_model2(train[i])
    acc_temp,val_temp=evaluate(words,test[i],pos,neg,one_count,zero_count)
    acc +=acc_temp
    val +=val_temp
    val_list.append(val_temp)
    acc_list.append(acc_temp)
val = val/5
acc = acc/5
print(acc)
val_arr = np.array(val_list)
var=np.var(val_arr)
print(val_arr)
# print(statistics.variance(val_list))
print("F_score -> "+str(val)+" +- "+str(statistics.stdev(val_list)))
print("Accuracy -> "+str(acc*100)+" +- "+str(statistics.stdev(acc_list)*100))

144
accuracy0.7164179104477612
precision0.6842105263157895
recall0.7142857142857143
f_Score0.6989247311827957
154
accuracy0.7661691542288557
precision0.7339449541284404
recall0.8247422680412371
f_Score0.7766990291262136
156
accuracy0.7761194029850746
precision0.728
recall0.900990099009901
f_Score0.8053097345132744
163
accuracy0.8109452736318408
precision0.8173913043478261
recall0.8545454545454545
f_Score0.8355555555555555
164
accuracy0.8159203980099502
precision0.7981651376146789
recall0.8613861386138614
f_Score0.8285714285714286
0.7771144278606965
[0.69892473 0.77669903 0.80530973 0.83555556 0.82857143]
F_score -> 0.7890120957898537 +- 0.055371907041279235
Accuracy -> 77.71144278606965 +- 4.017239649390318
